In [ ]:
from transformers import pipeline
import random
from sentence_transformers import SentenceTransformer
import numpy as np
import nltk  # Natural Language Toolkit
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Initialize Llama2 model from Hugging Face
llama2 = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')

# Initialize sentence transformer model for embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Task Definition - Here's a general task for creative writing
task = "Write a short story about a lost kitten"

# Initial Population
initial_population_size = 50
population = [llama2(task, max_length=50, do_sample=True)[0]['generated_text'] for _ in range(initial_population_size)]

def dot_product_similarity(text1, text2):
    # Generate embeddings for each text
    embedding1 = model.encode(text1)
    embedding2 = model.encode(text2)
    
    # Compute the dot product of the two embeddings
    similarity = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    return similarity

# Fitness Function
def fitness(prompt, desired_output):
    generated_output = llama2(prompt, max_length=50)[0]['generated_text']
    # Using dot product similarity between embeddings for fitness
    return dot_product_similarity(generated_output, desired_output)

# Crossover Operation
def crossover(parent1, parent2):
    # Simple one-point crossover
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return ' '.join(child1), ' '.join(child2)

# Function to tokenize and represent prompts as a list of words
def represent_prompt(prompt):
    return word_tokenize(prompt)

# Mutation Operation
def mutate_prompt(prompt):
    words = represent_prompt(prompt)
    if words:
        idx = random.randint(0, len(words) - 1)
        words[idx] = "adventure"  # changing a random word; you might want to select more sensibly
    return ' '.join(words)

# Selection Strategy
def select_population(population, desired_output):
    scored_population = [(prompt, fitness(prompt, desired_output)) for prompt in population]
    scored_population.sort(key=lambda x: x[1])  # Lower edit distance is better
    return [prompt for prompt, _ in scored_population[:initial_population_size]]

# Evolution Function
def evolve_population(population, desired_output):
    new_population = select_population(population, desired_output)
    while len(new_population) < len(population):
        parent1, parent2 = random.sample(new_population, 2)
        child1, child2 = crossover(represent_prompt(parent1), represent_prompt(parent2))
        new_population += [child1, child2]
    return [mutate_prompt(prompt) for prompt in new_population]

# Desired Output for the Task - This should be a specific example of a good output
desired_output = "Once upon a time, there was a small kitten named Whiskers who was lost in a big, dark forest."

# Run the genetic algorithm for a number of generations
generations = 20  # Number of generations
for _ in range(generations):
    population = evolve_population(population, desired_output)

# Select the best prompt
best_prompt = select_population(population, desired_output)[0]
print("Best prompt found:", best_prompt)
